In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
import subprocess
from bokeh.io.export import get_screenshot_as_png
import sqlite3
import time

output_notebook()

In [ ]:
database_1 = "./Data/test.db"
database_2 = "./Data/test2.db"

In [ ]:
conn = sqlite3.connect(database_1)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_names = cursor.fetchall()

cursor.close()
conn.close()

for name in table_names:
    print(name[0])


return_code = subprocess.check_output(["./sqldiff.exe", database_1, database_2])
output = return_code.decode("utf-8").splitlines()
inserts = [line for line in output if line.startswith("INSERT")]
updates = [line for line in output if line.startswith("UPDATE")]
deletes = [line for line in output if line.startswith("DELETE")]

#print({'inserts': inserts, 'updates': updates, 'deletes': deletes})

p = figure(x_range=['inserts', 'updates', 'deletes'],  title="Change in records between two databases")
p.vbar(x=['inserts', 'updates', 'deletes'], top=[len(inserts), len(updates), len(deletes)], width=0.9)

t = show(p,  notebook_handle=True)

In [ ]:
tables_altered = {'updated':[], 'insertions':[], 'deletions':[]}
total_tables_altered = set()

for i in range(len(updates)):
    x = updates[i].split()[1]
    print(x)
    tables_altered.get('updated').append(x)
    total_tables_altered.add(x)

for i in range(len(inserts)):
    x = inserts[i].split()[2].split('(')[0]
    print(x)
    tables_altered.get('insertions').append(x)
    total_tables_altered.add(x)

for i in range(len(deletes)):
    x = deletes[i].split()[2]
    print(x)
    tables_altered.get('deletions').append(x)
    total_tables_altered.add(x)

tables_altered

In [ ]:
inserts = inserts # * 1000

In [ ]:
table = inserts[0][12:].split('(')[0]
values_changed = inserts[0][12:].split('(')[1].split(')')[0].split(',')

table

In [ ]:
inserts[0][12:].split('(')[2:]

In [ ]:
values_changed = inserts[0][12:].split('(')
z = values_changed[1:]
# Parse this list until we hit values to get genre ids
falsify = True
idx = 0

while falsify and idx < len(z):
    if ' VALUES' in z[idx]:
        cur = z[idx+1:]
        
    idx += 1

values = ''.join(cur)[:-2].split(',')
print(values)

In [ ]:
start_time = time.time()
conn = sqlite3.connect(database_2)
cursor = conn.cursor()

insert_data = []

for i in range(len(inserts)):
    table = inserts[i][12:].split('(')[0]
   
    values_changed = inserts[i][12+len(table) + 1:]
    values_changed = values_changed[:values_changed.find(') VALUES')]
    z = values_changed[1:]
    falsify = True
    idx = 0
    while falsify and idx < len(z):
        if ' VALUES' in z[idx]:
            cur = z[idx+1:]
            
        idx += 1
    
    values = ''.join(cur)[:-2].split(',')
    print(f"{values} of type {values_changed} into {table}")
    insert_data.append((table, values, values_changed))

cursor.close()
conn.close()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
conn = sqlite3.connect(database_1)
cursor = conn.cursor()

delete_data = []

for i in range(len(deletes)):
    rest = deletes[i][12:]
    table = rest.split()[0]
    rest = rest[len(table) + 1:]
    primaryKey, ID = rest.split()[1].split('=')
    ID = ID[:-1]
    print('Deleted ', primaryKey, ':',  ID)
    cursor.execute(f"SELECT * FROM {table} WHERE {primaryKey}={ID};")
    print(cursor.fetchall())
    delete_data.append([table, primaryKey, ID])

cursor.close()
conn.close()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
conn = sqlite3.connect(database_1)
cursor = conn.cursor()
conn2 = sqlite3.connect(database_2)
cursor2 = conn2.cursor()

update_data = []

for i in range(len(updates)):
    table = updates[i].split()[1]
    print(table)
    Primarykey, ID = updates[i].split('WHERE ')[-1][:-1].split('=')
    print('Updated:', Primarykey, ID)
    rest = updates[i][updates[i].find('SET') + 4:]
    rest[:rest.find(' WHERE ')]
    cursor.execute(f"SELECT * FROM {table} WHERE {Primarykey}={ID};")
    print(cursor.fetchall())
    cursor2.execute(f"SELECT * FROM {table} WHERE {Primarykey}={ID};")
    print(cursor2.fetchall())
    update_data.append([table, Primarykey, ID])

cursor.close()
conn.close()
cursor2.close()
conn2.close()

In [ ]:
insert_data, delete_data, update_data

In [ ]:
import pandas as pd

df = pd.DataFrame(insert_data, columns=['Table', 'Primary Key', 'ID'])
df['type'] = 'insert'
df2 = pd.DataFrame(delete_data, columns=['Table', 'Primary Key', 'ID'])
df2['type'] = 'delete'
df3 = pd.DataFrame(update_data, columns=['Table', 'Primary Key', 'ID'])
df3['type'] = 'update'

df = pd.concat([df, df2, df3], ignore_index=True)
df